In [5]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import glob
import sys
from astroquery.jplhorizons import Horizons

In [6]:
### Change as required

main_dir = '/Users/s1523386/Documents/year1/shape_modelling/162p'
lc_dir = main_dir+'/LC_prep/raw/'
target = '162p'

files = sorted(glob.glob(lc_dir+target+'*R.txt'))
# Check this read correctly
if len(files)==0:
    print('Problem loading LC files, check path.')
    sys.exit()
else:
    print('%i LC files found for %s.'%(len(files), target))
    print('First file in list = ', files[0])
    print('Last file in list = ', files[-1])

18 LC files found for 162p.
First file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20070517_950_R.txt
Last file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20180711_809_R.txt


In [7]:
# Magnitudes found in the second column of each LC file, JD in first
# Calculate mean magnitude from each LC as it's read in
mjd_start = []
mjd_all=[]
mag = []
mean_mag = []
mag_all = []
mag_unc=[]
mjd = []
length_list = []
npoints=np.zeros(len(files))

for i,file in enumerate(files):
    
    count = 0
    f = open(file, 'r')
    lines = f.readlines()
    for line in lines:
        data = line.split()
        mjd_all.append(float(data[0]))
        mjd.append(float(data[0]))
        mag.append(float(data[1]))
        mag_all.append(float(data[1]))
        mag_unc.append(float(data[2]))
        count = count+1
    npoints[i] = int(count)
    mjd_start.append(mjd[0])    # MJDs at start of observing run (mid-exposure)
    print('No. mags to take mean of =', len(mag))
    length_list.append(len(mag))
    mean_mag.append(np.mean(mag))
    mjd = []
    mag = []
    
f.close()

No. mags to take mean of = 13
No. mags to take mean of = 12
No. mags to take mean of = 12
No. mags to take mean of = 30
No. mags to take mean of = 5
No. mags to take mean of = 17
No. mags to take mean of = 13
No. mags to take mean of = 29
No. mags to take mean of = 93
No. mags to take mean of = 52
No. mags to take mean of = 79
No. mags to take mean of = 21
No. mags to take mean of = 80
No. mags to take mean of = 71
No. mags to take mean of = 51
No. mags to take mean of = 17
No. mags to take mean of = 15
No. mags to take mean of = 8


In [8]:
# Obtain telescope codes from files filenames to parse Horizons database
siteid = []
for file in files:
    siteid.append(file[79:82])
print(siteid) # Check this is a list of 3-digit numbers 


['950', '950', '950', '309', '309', '809', '809', '309', '950', '950', '950', '071', '950', '950', '950', '809', '809', '809']


In [9]:
# Correct mid-exposure MJDs for light travel time before we obtain alphas, deltas and r_hs
# MJD_use = MJD_start-(delta/c)
au=1.496e11   #m
d2s=86400
mjd_use = []

for i,day in enumerate(mjd_start):
    obj = Horizons(id='90001062',location = siteid[i], epochs=day)
    eph = obj.ephemerides(quantities='20')
    deltcomet = float(eph['delta'])
    mjd_use.append(((day*d2s)-((deltcomet*au)/3.e8))/d2s)
mjd_use

[2454238.4504419076,
 2454239.36251031,
 2454240.3729025465,
 2456040.571266327,
 2456071.4737502034,
 2456093.490967207,
 2456096.4932989585,
 2456101.438880528,
 2457802.5503569725,
 2457803.5527208317,
 2457806.5423623323,
 2457811.504141046,
 2458219.455531825,
 2458220.4885629457,
 2458227.435531547,
 2458309.3739130935,
 2458310.374203054,
 2458311.370768656]

In [12]:
# Use light-time corrected MJDs to parse Horizons for delta, alpha, r_h
r_au = []
delta_au = []
alpha = []

for i in range(len(siteid)):
    obj = Horizons(id='90001066',location = siteid[i], epochs=mjd_use[i])
    eph = obj.ephemerides(quantities='19,20,24')
    
    r = float(eph['r'])
    r_au.append(r)
    
    delta = float(eph['delta'])
    delta_au.append(delta)
    alp = float(eph['alpha'])
    alpha.append(alp)

In [13]:
# Check the ephemerides are for 162P/Siding-Spring: it's changed id x3 since I started this...
print(eph)

    targetname           datetime_str       ... delta_rate  alpha 
       ---                   ---            ...   km / s     deg  
------------------ ------------------------ ... ---------- -------
162P/Siding Spring 2018-Jul-11 20:53:54.412 ... 26.0869665 12.0887


In [27]:
print(len(r_au), len(delta_au), len(alpha))

18 18 18


In [43]:
# Defining funciton to calculate reduced magnitude H(1,1,alpha)
# Inputs:
#    m: apparent magnitude (t = tE)
#    r: heliocentric distance (AU) (t = t0)
#    delta: geocentric distance (AU) (t = t0)

# Outputs:
#    h: H(1,1,alpha)
def mag_reduce(m, r, delta):
    h = m-(5*np.log10(r*delta))
    return h

In [42]:
dir_name = main_dir+'/LC_prep/dist_corr'
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

count = 0       
for i,day in enumerate(mjd_all):
    if (day in mjd_start):
        #fout.close()
        count = count+1
        #fout = open(dir_name+files[count-1][64:],'w+')
        print(count, '%.6f' % day, '%.3f' %mag_reduce(mag_all[i], r_au[count-1], delta_au[count-1]))#, mag_unc[i])
        #fout.write(str('%.6f' % day)+' '+str('%.3f' %mag_reduce(mag[i], r_au[count-1], delta_au[count-1]))+' '+str(mag_unc[i])+'\n')
    else:
        print(count, '%.6f' % day, '%.3f' %mag_reduce(mag_all[i], r_au[count-1], delta_au[count-1]))#, mag_unc[i])
        #fout.write(str('%.6f' % day)+' '+str('%.3f' % mag_reduce(mag[i], r_au[count-1], delta_au[count-1]))+' '+str(mag_unc[i])+'\n')
#fout.close()   

1 2454238.473696 13.966
1 2454238.475177 13.970
1 2454238.476670 13.949
1 2454238.478152 13.959
1 2454238.479633 13.952
1 2454238.481115 13.941
1 2454238.482596 13.986
1 2454238.484078 13.939
1 2454238.485582 14.000
1 2454238.487064 13.964
1 2454238.549216 13.974
1 2454238.553696 13.981
1 2454238.558152 13.982
2 2454239.385819 14.235
2 2454239.387543 14.204
2 2454239.389256 14.240
2 2454239.390993 14.206
2 2454239.392705 14.195
2 2454239.399418 14.232
2 2454239.404476 14.300
2 2454239.551745 14.412
2 2454239.553469 14.395
2 2454239.601224 14.411
2 2454239.602717 14.328
2 2454239.604198 14.375
3 2454240.396273 14.080
3 2454240.397766 14.150
3 2454240.399247 14.137
3 2454240.471122 13.961
3 2454240.472604 13.976
3 2454240.474131 13.986
3 2454240.546724 13.981
3 2454240.548205 13.996
3 2454240.549687 13.983
3 2454240.607638 13.986
3 2454240.609120 13.962
3 2454240.610613 13.962
4 2456040.641995 13.847
4 2456040.643064 13.829
4 2456040.644129 13.851
4 2456040.645204 13.856
4 2456040.646293

18 2458311.451651 14.312
18 2458311.454363 14.286
18 2458311.457068 14.238
18 2458311.479609 14.198
18 2458311.483478 14.229
18 2458311.487346 14.245
18 2458311.561172 14.168
18 2458311.569139 14.203


In [37]:
print(mag)

[]


In [23]:
# Check solar phase angles are sensible
alpha

[7.5057,
 7.6664,
 7.8417,
 4.4883,
 9.892,
 11.8385,
 11.97,
 12.1195,
 9.8697,
 9.6987,
 9.1704,
 8.2387,
 2.2635,
 2.0222,
 0.3947,
 12.0582,
 12.0751,
 12.0887]

In [1]:
#mjd_all

In [29]:
# Briefly experimenting with convexinv output files 
# read in output lcs from convexinv
intensity=[]
path = '/Users/s1523386/Documents/year1/shape_modelling/162p/dist_cal_pole_scan/run_0_-90_2021_05_04/out_lcs'

filenom = path

g = open(filenom, 'r')
lines = g.readlines()

for line in lines:
    data = line.split()
    intensity.append(float(data[0]))

In [2]:
#intensity